In [41]:
spark.stop()

In [42]:
#0 Docker, 1 CloudVeneto
FIGHTER = 1
core = 4
nome = "marco"
#nicolò
#marco
#francesco
#raffaele

In [43]:
# mapping utenti → (SparkUI, user)
mappa = {
    "nicolò":   (4040, "user_a"),
    "marco":    (4041, "user_b"),
    "francesco":(4042, "user_c"),
    "raffaele": (4043, "user_d"),
}
# recupero SparkUI e user dal mapping
SparkUI, user = mappa[nome]
Npartition = 4*core # Regola generale: numero_partizioni = numero_core * (2 o 4)

print(f"Hai settato {SparkUI} e {user}: tu sei {nome}")
Npartition = 8

Hai settato 4041 e user_b: tu sei marco


In [44]:
import os
import numpy as np
import pandas as pd
from math import ceil
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

from pyspark import SparkFiles
from pyspark.sql import SparkSession, DataFrame, Window
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, TimestampType, BooleanType, StringType

from pyspark.sql.functions import (
    coalesce, corr, array,
    col, lit, expr, when, count, count_if, row_number, sum as spark_sum, abs as spark_abs,
    round as spark_round, min as spark_min, max as spark_max, avg as spark_avg,
    first, last, lag, lead, row_number, desc, asc, bool_or, floor,
    explode, sequence, from_unixtime, to_date, unix_timestamp,
    window, min_by, mode, concat, monotonically_increasing_id
)

import datetime

In [45]:
if FIGHTER==0:

    spark = SparkSession.builder \
        .master("spark://spark-master:7077") \
        .appName("ProjectDocker") \
        .config("spark.executor.memory", "1000m") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false") \
        .getOrCreate()

        # SE NON FUNZIONA TOGLI I DUE CONFIG DI ARROW

        # .config("spark.executor.memory", "1500m")
        # .config("spark.executor.cores", "1")\
        # .config("spark.executor.instances", "12")\
        # .config("spark.cores.max", "12")\
        # .config("spark.default.parallelism", "24")\
        # .config("spark.sql.adaptive.enabled", "true")\

elif FIGHTER==1:

        os.environ["PYSPARK_PYTHON"] = "/opt/miniconda3/bin/python"
        os.environ["PYSPARK_DRIVER_PYTHON"] = "/opt/miniconda3/bin/python"
        
        spark = SparkSession.builder \
            .appName("ProjectCloudVeneto") \
            .master("spark://10.67.22.135:7077") \
            .config("spark.scheduler.mode", "FAIR") \
            .config("spark.scheduler.pool", user) \
            .config("spark.scheduler.allocation.file", "file:///usr/local/spark/conf/fairscheduler.xml") \
            .config("spark.cores.max", core) \
            .config("spark.executor.memory", "800m") \
            .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
            .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false") \
            .config("spark.dynamicAllocation.enabled", "false") \
            .config("spark.shuffle.service.enabled", "false") \
            .config("spark.ui.port", SparkUI) \
            .getOrCreate()
    
else : print("Better choose an available fighter, you little bastard.")

sc = spark.sparkContext

In [35]:
# if FIGHTER==0:
#     df = spark.read.option("header", True).option("inferSchema", True).csv("/ProvePreliminari/SW-106.csv")

# elif FIGHTER==1:
#     df = spark.read.option("header", True).option("inferSchema", True).csv("file:///mnt/shared/dataset.csv")

# else:
#     print("Better choose an available fighter, you little bastard")

# df = df.repartition(Npartition)

# Unixtime (utile per controllare)

In [39]:
from datetime import datetime
print ( datetime.fromtimestamp(1601510430) )
print ( datetime.fromtimestamp(1601510550) )

2020-10-01 00:00:30
2020-10-01 00:02:30


# expr

In [31]:
data = [
    ("window1", 1, 10),
    ("window1", 0, 20),
    ("window2", 1, 30),
    ("window2", 1, 40),
]
columns = ["time_window", "A5", "A7"]

df_1 = spark.createDataFrame(data, columns)

print("DataFrame iniziale:")
df_1.show()


df_2 = df_1.agg(spark_avg(col("A7")).alias("media A7"))  #di fatto sottointeso .goupBy su tutto il dataframe
df_2.show()


df_3 = df_1.groupBy("time_window").agg(spark_avg(col("A7")).alias("media A7"))
df_3.show()

DataFrame iniziale:
+-----------+---+---+
|time_window| A5| A7|
+-----------+---+---+
|    window1|  1| 10|
|    window1|  0| 20|
|    window2|  1| 30|
|    window2|  1| 40|
+-----------+---+---+

+--------+
|media A7|
+--------+
|    25.0|
+--------+

+-----------+--------+
|time_window|media A7|
+-----------+--------+
|    window1|    15.0|
|    window2|    35.0|
+-----------+--------+



# agg

In [21]:
# 2. Creare il DataFrame di esempio
data = [
    ("window1", 1, 10),
    ("window1", 0, 20),
    ("window2", 1, 30),
    ("window2", 1, 40),
]
columns = ["time_window", "A5", "A7"]

df_windowed = spark.createDataFrame(data, columns)

print("DataFrame iniziale:")
df_windowed.show()

# 3. Definire le aggregazioni
aggs = [
    spark_max(col("A5")).alias("A5"),   # MAX per A5
    spark_avg(col("A7")).alias("A7")    # AVG per A7
]

# 4. Eseguire la groupBy con le aggregazioni
result_df = df_windowed.groupBy("time_window").agg(*aggs)

print("Risultato aggregato:")
result_df.show()

DataFrame iniziale:
+-----------+---+---+
|time_window| A5| A7|
+-----------+---+---+
|    window1|  1| 10|
|    window1|  0| 20|
|    window2|  1| 30|
|    window2|  1| 40|
+-----------+---+---+

Risultato aggregato:
+-----------+---+----+
|time_window| A5|  A7|
+-----------+---+----+
|    window2|  1|35.0|
|    window1|  1|15.0|
+-----------+---+----+



# window

In [11]:
# 2. Creiamo un DataFrame con timestamp + valori
data = [
    ("user1", 10, datetime.datetime(2023, 1, 1, 12, 0, 0)),
    ("user2", 20, datetime.datetime(2023, 1, 1, 12, 3, 0)),
    ("user1", 30, datetime.datetime(2023, 1, 1, 12, 7, 0)),
    ("user2", 40, datetime.datetime(2023, 1, 1, 12, 12, 0)),
    ("user1", 50, datetime.datetime(2023, 1, 1, 12, 18, 0)),
]

schema = StructType([
    StructField("user", StringType(), True),
    StructField("value", IntegerType(), True),
    StructField("timestamp", TimestampType(), True),
])

df = spark.createDataFrame(data, schema)

print("DataFrame originale:")
df.show(truncate=False)

# 3. Applichiamo una window di 10 minuti sul timestamp
df_windowed = (
    df.groupBy(window(col("timestamp"), "10 minutes"))  # ogni 10 minuti
      .agg(
          spark_avg("value").alias("avg_value"),
          count("*").alias("count")
      )
      .orderBy("window")
)

print("Aggregato per finestre di 10 minuti:")
df_windowed.show(truncate=False)

DataFrame originale:
+-----+-----+-------------------+
|user |value|timestamp          |
+-----+-----+-------------------+
|user1|10   |2023-01-01 12:00:00|
|user2|20   |2023-01-01 12:03:00|
|user1|30   |2023-01-01 12:07:00|
|user2|40   |2023-01-01 12:12:00|
|user1|50   |2023-01-01 12:18:00|
+-----+-----+-------------------+

Aggregato per finestre di 10 minuti:


[Stage 4:============================================>              (3 + 1) / 4]

+------------------------------------------+---------+-----+
|window                                    |avg_value|count|
+------------------------------------------+---------+-----+
|{2023-01-01 12:00:00, 2023-01-01 12:10:00}|20.0     |3    |
|{2023-01-01 12:10:00, 2023-01-01 12:20:00}|45.0     |2    |
+------------------------------------------+---------+-----+



In [14]:
# 2. Piccolo dataset con timestamp
data = [
    (1, datetime.datetime(2023, 1, 1, 12, 0, 0)),
    (2, datetime.datetime(2023, 1, 1, 12, 3, 0)),
    (3, datetime.datetime(2023, 1, 1, 12, 7, 0)),
    (4, datetime.datetime(2023, 1, 1, 12, 12, 0)),
]

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("timestamp", TimestampType(), True),
])

df = spark.createDataFrame(data, schema)

print("DataFrame originale:")
df.show(truncate=False)

# 3. Usiamo SOLO la funzione window
df_windowed = df.select(window(col("timestamp"), "10 minutes").alias("time_window"))

print("DataFrame con le finestre temporali:")
df_windowed.show(truncate=False)

DataFrame originale:
+---+-------------------+
|id |timestamp          |
+---+-------------------+
|1  |2023-01-01 12:00:00|
|2  |2023-01-01 12:03:00|
|3  |2023-01-01 12:07:00|
|4  |2023-01-01 12:12:00|
+---+-------------------+

DataFrame con le finestre temporali:
+------------------------------------------+
|time_window                               |
+------------------------------------------+
|{2023-01-01 12:00:00, 2023-01-01 12:10:00}|
|{2023-01-01 12:00:00, 2023-01-01 12:10:00}|
|{2023-01-01 12:00:00, 2023-01-01 12:10:00}|
|{2023-01-01 12:10:00, 2023-01-01 12:20:00}|
+------------------------------------------+



In [ ]:
# {2023-01-01 12:00:00, 2023-01-01 12:10:00}
# in realtà è una struct con due campi: start e end.

In [18]:
df_estremi = df_windowed.select(col("time_window.start").alias("inizio"),col("time_window.end").alias("fine"))
df_estremi.show()

+-------------------+-------------------+
|             inizio|               fine|
+-------------------+-------------------+
|2023-01-01 12:00:00|2023-01-01 12:10:00|
|2023-01-01 12:00:00|2023-01-01 12:10:00|
|2023-01-01 12:00:00|2023-01-01 12:10:00|
|2023-01-01 12:10:00|2023-01-01 12:20:00|
+-------------------+-------------------+



# lit()

In [ ]:
# lit(1) → crea una colonna con il valore costante 1 in tutte le righe.
# lit("ciao") → colonna costante con "ciao"

In [41]:
data = [("Alice", 25), ("Bob", 30)]
columns = ["name", "age"]

df = spark.createDataFrame(data, columns)

print("DataFrame originale:")
df.show()

# Aggiungiamo una colonna costante con lit(1)
df_with_flag = df.withColumn("flag-lit1-", lit(1))
df_prova = df_with_flag.withColumn("flag-ciao", lit("ciao"))

df_prova.show()

# PartitonBy DI WINDOW (OCCHIO IMPORTANTE SIA DI WINDOW il partition)

In [ ]:
data = [
    ("Alice", 10),
    ("Alice", 20),
    ("Bob", 5),
    ("Bob", 15)
]

df = spark.createDataFrame(data, ["name", "score"])
print("DataFrame originale:")
df.show()

# 1️⃣ Window globale usando lit(1)
w_global = Window.partitionBy(lit(1)).orderBy("score")
df_global = df.withColumn("row_num_global", row_number().over(w_global))

print("Row number con partitionBy(lit(1)) (tutti in un unico gruppo):")
df_global.show()

# 2️⃣ Window per gruppo usando name
w_by_name = Window.partitionBy("name").orderBy("score")
df_by_name = df.withColumn("row_num_by_name", row_number().over(w_by_name))

print("Row number con partitionBy('name') (numerazione separata per ogni name):")
df_by_name.show()

Come funziona realmente partitionBy in una Window

Non sposta fisicamente i dati sul disco
Quando scrivi

w = Window.partitionBy("name").orderBy("score")
df.withColumn("row_num", row_number().over(w))

Spark non riscrive fisicamente il DataFrame in partizioni distinte sul cluster.
Internamente crea solo una logica di raggruppamento temporaneo in memoria per applicare le funzioni di finestra.
È un espediente logico per le window functions
Spark sa che per calcolare row_number, rank, lag, ecc., deve trattare insieme tutte le righe con lo stesso valore di partitionBy.
Poi le ordina secondo orderBy all’interno di ciascuna “partizione logica” per calcolare la funzione.

Quando il dati vengono fisicamente spostati? ---- QUESTO E' IL PARTITION BY DEL DATAFRAME, NON DI WINDOW -----
Solo se fai operazioni come repartition("name") o write.partitionBy("name").
In quel caso Spark crea partizioni fisiche distinte, cioè i dati vengono distribuiti su più nodi o file separati.

# Break lineage (da approfondire)

In [ ]:
df_anom = spark.createDataFrame(df_anom.rdd, df_anom.schema)  # Break lineage

# Anomaly

In [5]:
def createDummyAnomaly(nBlocks, feature_name, values):
    
    schema = StructType([
        StructField("BlockID", IntegerType(), True),
        StructField("window_id", IntegerType(), True),
        StructField("when", IntegerType(), True),
        StructField(feature_name, IntegerType(), True)
    ])

    data = []

    for k in range(nBlocks):
        for n in range(len(values)):
            data.append((k, n, k*len(values)+n*60, values[n]))

    df = spark.createDataFrame(data, schema)
    return df.orderBy('BlockID','when')

In [8]:
dummy_df = createDummyAnomaly(2, 'S117', [0,0,1,1,1,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,1,1,1])
dummy_df.show(30)
print( f"numero righe  {dummy_df.count()}" )

+-------+---------+----+----+
|BlockID|window_id|when|S117|
+-------+---------+----+----+
|      0|        0|   0|   0|
|      0|        1|  60|   0|
|      0|        2| 120|   1|
|      0|        3| 180|   1|
|      0|        4| 240|   1|
|      0|        5| 300|   0|
|      0|        6| 360|   0|
|      0|        7| 420|   0|
|      0|        8| 480|   0|
|      0|        9| 540|   1|
|      0|       10| 600|   1|
|      0|       11| 660|   0|
|      0|       12| 720|   1|
|      0|       13| 780|   0|
|      0|       14| 840|   1|
|      0|       15| 900|   0|
|      0|       16| 960|   0|
|      0|       17|1020|   0|
|      0|       18|1080|   0|
|      0|       19|1140|   1|
|      0|       20|1200|   1|
|      0|       21|1260|   1|
|      0|       22|1320|   1|
|      1|        0|  23|   0|
|      1|        1|  83|   0|
|      1|        2| 143|   1|
|      1|        3| 203|   1|
|      1|        4| 263|   1|
|      1|        5| 323|   0|
|      1|        6| 383|   0|
+-------+-

# Lag / Lead

In [27]:
# Creo un DataFrame di esempio
data = [
    (1, "A", 100),
    (2, "A", 200),
    (3, "A", 300),
    (4, "B", 400),
    (5, "B", 500),
    (6, "B", 600),
]

df = spark.createDataFrame(data, ["id", "group", "value"])

# Definisco una finestra per "group" ordinata per "id"
windowSpec = Window.partitionBy("group").orderBy("id")


# Applico la funzione lag per ottenere il valore precedente
df_with_lag = df.withColumn("prev_value", lead("value", 1).over(windowSpec)) # lag or lead --- cambia  qui

# Stampo il tipo e la rappresentazione della windowSpec
print("Tipo:", type(windowSpec))
print("Rappresentazione:", windowSpec)

df_with_lag.show()

Tipo: <class 'pyspark.sql.window.WindowSpec'>
Rappresentazione: <pyspark.sql.window.WindowSpec object at 0x7f6fc7e80280>
+---+-----+-----+----------+
| id|group|value|prev_value|
+---+-----+-----+----------+
|  1|    A|  100|       200|
|  2|    A|  200|       300|
|  3|    A|  300|      NULL|
|  4|    B|  400|       500|
|  5|    B|  500|       600|
|  6|    B|  600|      NULL|
+---+-----+-----+----------+



In [25]:
sensors = ["S117"]
window         = Window.partitionBy("BlockID").orderBy("window_id")
lagged_columns = [lag(col(s)).over(window) for s in sensors] 
lag_names      = [f"lagged_{s}" for s in sensors]

df_interest = dummy_df.select("BlockID","window_id","when",*sensors)
df_lagged = df_interest.withColumns(dict(zip(lag_names, lagged_columns)))
df_lagged.show(30)

print( type(lagged_columns) )
switch_columns = [ when(  (col(s) != col(f"lagged_{s}")) ,1 ).otherwise(0) for s in sensors] 
switch_names      = [f"haSwitchato_{s}" for s in sensors]

df_lag_switch = df_lagged.withColumns(dict(zip( switch_names, switch_columns)))
df_lag_switch.show(30)

+-------+---------+----+----+-----------+
|BlockID|window_id|when|S117|lagged_S117|
+-------+---------+----+----+-----------+
|      0|        0|   0|   0|       NULL|
|      0|        1|  60|   0|          0|
|      0|        2| 120|   1|          0|
|      0|        3| 180|   1|          1|
|      0|        4| 240|   1|          1|
|      0|        5| 300|   0|          1|
|      0|        6| 360|   0|          0|
|      0|        7| 420|   0|          0|
|      0|        8| 480|   0|          0|
|      0|        9| 540|   1|          0|
|      0|       10| 600|   1|          1|
|      0|       11| 660|   0|          1|
|      0|       12| 720|   1|          0|
|      0|       13| 780|   0|          1|
|      0|       14| 840|   1|          0|
|      0|       15| 900|   0|          1|
|      0|       16| 960|   0|          0|
|      0|       17|1020|   0|          0|
|      0|       18|1080|   0|          0|
|      0|       19|1140|   1|          0|
|      0|       20|1200|   1|     

# Bool_or oppure spark_max / spark_min

In [48]:
data = [
    ("A", True),
    ("A", False),
    ("A", False),
    ("B", False),
    ("B", False),
    ("B", True)
]

df = spark.createDataFrame(data, ["group", "flag"])

# bool_or: almeno un True per gruppo
df_bool_or = df.groupBy("group").agg(bool_or("flag").alias("any_true"))
df.show()
df_bool_or.show()

+-----+-----+
|group| flag|
+-----+-----+
|    A| true|
|    A|false|
|    A|false|
|    B|false|
|    B|false|
|    B| true|
+-----+-----+

+-----+--------+
|group|any_true|
+-----+--------+
|    A|    true|
|    B|    true|
+-----+--------+



In [49]:
data = [
    ("A", 10),
    ("A", 20),
    ("A", 30),
    ("B", 40),
    ("B", 50)
]

df = spark.createDataFrame(data, ["group", "value"])

# Calcoliamo la media per ogni gruppo
df_avg = df.groupBy("group").agg(spark_max("value").alias("avg_value"))

df.show()
df_avg.show()

+-----+-----+
|group|value|
+-----+-----+
|    A|   10|
|    A|   20|
|    A|   30|
|    B|   40|
|    B|   50|
+-----+-----+

+-----+---------+
|group|avg_value|
+-----+---------+
|    B|       50|
|    A|       30|
+-----+---------+



# Close Spark session

In [34]:
spark.stop()